# Database import jupyter script

## I. Setup

### 1. Django Jupyter setup

In [1]:
import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### 2. Environment variables

In [2]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

### 3. Other imports

In [3]:
from account.models import *
from sales.models import *

## II. Import

### 1. Special Offers

In [ ]:
specialOfferList = SpecialOffer.objects.all()
print(specialOfferList)